In [1]:
import sys
sys.path.append("/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL")

In [23]:
# Import Libraries
import time
import sklearn
import numpy as np
np.random.seed(0)
import pandas as pd
from topologicpy.DGL import DGL
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
import itertools
import dgl
import torch
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import random
random.seed(0)

import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
from dgl import save_graphs, load_graphs
from tqdm.notebook import tqdm

In [24]:
path = "./data3"
dataset = DGL.DatasetByCSVPath(path, numberOfGraphClasses=0,
                           nodeATTRKey='feat', edgeATTRKey='feat',
                           nodeOneHotEncode=False, nodeFeaturesCategories=[],
                           edgeOneHotEncode=False, edgeFeaturesCategories=[], addSelfLoop=False)
print(dataset)

Done saving data into cached files.
Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)


In [26]:
train_data, val_data, test_data = dgl.data.utils.split_dataset(dataset, [0.7, 0.1, 0.2], random_state=101)

In [27]:
train_data.dataset

Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)

---

In [28]:
# Contruct a two-layer GNN model
# import dgl.nn as dglnn
# import torch.nn as nn
# import torch.nn.functional as F
# class SAGE(nn.Module):
#     def __init__(self, in_feats, hid_feats, out_feats):
#         super().__init__()
#         self.conv1 = dglnn.SAGEConv(
#             in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
#         self.conv2 = dglnn.SAGEConv(
#             in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

#     def forward(self, graph, inputs):
#         # inputs are features of nodes
#         h = self.conv1(graph, inputs)
#         h = F.relu(h)
#         h = self.conv2(graph, h)
#         return h

In [29]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [30]:
# # print(len(dataset.graphs))
# for i, graph in enumerate(dataset.graphs):
#     # print(i)
#     node_features = graph.ndata['feat']
#     # print(node_features)
#     node_labels = graph.ndata['label']
#     # print(node_labels)
#     train_mask = graph.ndata['train_mask']
#     valid_mask = graph.ndata['val_mask']
#     test_mask = graph.ndata['test_mask']
#     n_features = node_features.shape[1]
#     # print(n_features)
#     n_labels = int(node_labels.max().item() + 1)
#     # print(n_features)

# Grid Search

In [31]:
# num_hidden_layers = [2, 3, 4, 5]
# num_units_per_layer = [25, 50, 100]
# activation_functions = [F.relu, F.tanh]
# learning_rates = [0.01, 0.001, 0.0005]
# batch_sizes = [16, 32, 64]
# aggregator_type = ['mean', 'pool', 'lstm']

In [32]:
def evaluate_selected_model(model):
    dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
    
    actual = []
    for l in dictionary['alllabels']:
        actual.extend(l)
        
    predicted = []
    for l in dictionary['allpredictions']:
        predicted.extend(l)

    result = DGL.Accuracy(actual=actual, predicted=predicted)
    accuracy = result['accuracy']
    precision = DGL.Precision(actual=actual, predicted=predicted)
    recall = DGL.Recall(actual=actual, predicted=predicted)
   
    return accuracy, precision, recall

class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, num_layers, activation, agg_type):
        super().__init__()
        self.layers = nn.ModuleList()
        self.activation = activation
        
        # Create the specified number of hidden layers
        self.layers.append(dglnn.SAGEConv(in_feats=in_feats, out_feats=hid_feats, aggregator_type=agg_type))
        for _ in range(num_layers - 1):
            self.layers.append(dglnn.SAGEConv(in_feats=hid_feats, out_feats=hid_feats, aggregator_type=agg_type))
        self.layers.append(dglnn.SAGEConv(in_feats=hid_feats, out_feats=out_feats, aggregator_type=agg_type))

    def forward(self, graph, inputs):
        h = inputs
        for layer in self.layers[:-1]:
            h = layer(graph, h)
            h = self.activation(h)  # Apply activation function
        h = self.layers[-1](graph, h)  # Last layer without activation
        return h


def train_with_grid(param_grid):
    results = []
    
    # Iterate over all combinations of hyperparameters
    for num_layers, hid_feats, activation, lr, batch_size, agg_type in tqdm(param_grid):
        print(f"Training with layers: {num_layers}, units per layer: {hid_feats}, activation: {activation.__name__}, learning rate: {lr}, batch size: {batch_size}, aggreagator type: {agg_type}")
        
        # Initialize model and optimizer
        n_features = 8
        model = SAGE(in_feats=n_features, hid_feats=hid_feats, out_feats=13, num_layers=num_layers, activation=activation, agg_type=agg_type)
        opt = torch.optim.Adam(model.parameters(), lr=lr)
        
        
    
        # for i, graph in enumerate(dataset.graphs):
        all_graph = dgl.batch(dataset.graphs)
            
        df = pd.DataFrame(columns=['epoch', 'loss', 'val_acc', 'time'])
        for epoch in range(100):
            start = time.time()        
            node_features = all_graph.ndata['feat']
            node_labels = all_graph.ndata['label']
            train_mask = all_graph.ndata['train_mask']
            valid_mask = all_graph.ndata['val_mask']
            test_mask = all_graph.ndata['test_mask']
            n_features = node_features.shape[1]
            n_labels = int(node_labels.max().item() + 1)

            model.train()
            # forward propagation by using all nodes
            logits = model(all_graph, node_features)
            # compute loss
            loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
            # compute validation accuracy
            acc = evaluate(model, all_graph, node_features, node_labels, valid_mask)
            # backward propagation
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_time = time.time() - start
            df.loc[len(df)]={"epoch": epoch, "loss": loss.item(), "val_acc": acc, "time": total_time}
            # print(loss.item())

        accuracy, precision, recall = evaluate_selected_model(model)  
        result = {
            'num_hidden_layers': num_layers,
            'num_units_per_layer': hid_feats,
            'activation_functions': activation,
            'learning_rates': lr, 
            'batch_sizes': batch_size,
            'aggregator_type': agg_type,
            'acc': accuracy,
            'precision': precision,
            'recall': recall,
            'details': df,
        }
        results.append(result)
    return results, model

In [35]:
num_hidden_layers = [2, 3, 4, 5]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

hidden_layers_results = train_with_grid(param_grid)
hidden_layers_results

  0%|          | 0/4 [00:00<?, ?it/s]

Training with layers: 2, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 4, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 5, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 2,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.766644,
   'precision': 0.766644,
   'recall': 0.766644,
   'details':     epoch       loss   val_acc      time
   0       0  59.860317  0.015873  0.031590
   1       1  54.706005  0.015873  0.023355
   2       2  49.620918  0.015873  0.015324
   3       3  44.605789  0.015873  0.015284
   4       4  39.651905  0.015873  0.011985
   ..    ...        ...       ...       ...
   95     95   0.690167  0.767196  0.015713
   96     96   0.686648  0.768519  0.011838
   97     97   0.682857  0.771164  0.012922
   98     98   0.679197  0.769841  0.010443
   99     99   0.675901  0.765873  0.010659
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [36]:
num_hidden_layers = [3]
num_units_per_layer = [25, 50, 75, 100]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

num_units_results = train_with_grid(param_grid)
num_units_results

  0%|          | 0/4 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 25, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 75, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 100, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 25,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.709629,
   'precision': 0.709629,
   'recall': 0.709629,
   'details':     epoch       loss   val_acc      time
   0       0  59.457886  0.022487  0.018305
   1       1  54.581963  0.022487  0.016561
   2       2  49.826000  0.022487  0.022761
   3       3  45.246208  0.022487  0.016657
   4       4  40.770927  0.022487  0.018160
   ..    ...        ...       ...       ...
   95     95   1.498372  0.664021  0.018503
   96     96   1.461614  0.669312  0.012490
   97     97   1.426321  0.674603  0.012116
   98     98   1.392486  0.702381  0.013165
   99     99   1.361298  0.715608  0.013528
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [37]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu, F.tanh]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

activation_results = train_with_grid(param_grid)
activation_results

  0%|          | 0/2 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: tanh, learning rate: 0.001, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.775513,
   'precision': 0.775513,
   'recall': 0.775513,
   'details':     epoch       loss   val_acc      time
   0       0  24.124041  0.015873  0.030403
   1       1  16.623455  0.242063  0.017814
   2       2  15.042412  0.312169  0.017081
   3       3  12.995396  0.325397  0.019701
   4       4  10.572761  0.326720  0.017878
   ..    ...        ...       ...       ...
   95     95   0.650066  0.771164  0.017964
   96     96   0.646522  0.773810  0.016104
   97     97   0.643039  0.772487  0.017526
   98     98   0.639437  0.773810  0.016236
   99     99   0.635766  0.777778  0.022473
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [38]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.01, 0.005, 0.001, 0.0005]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

lr_results = train_with_grid(param_grid)
lr_results

  0%|          | 0/4 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.01, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.005, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.0005, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.01,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.906128,
   'precision': 0.906128,
   'recall': 0.906128,
   'details':     epoch       loss   val_acc      time
   0       0  87.685829  0.022487  0.032089
   1       1  18.355539  0.326720  0.025683
   2       2   8.858552  0.365079  0.020585
   3       3   8.592385  0.351852  0.023103
   4       4   6.233907  0.370370  0.025810
   ..    ...        ...       ...       ...
   95     95   0.269638  0.915344  0.019177
   96     96   0.267917  0.914021  0.023957
   97     97   0.266050  0.914021  0.018580
   98     98   0.262147  0.915344  0.014452
   99     99   0.260582  0.919312  0.015250
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.

In [39]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [16, 32, 64]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

batch_results = train_with_grid(param_grid)
batch_results

  0%|          | 0/3 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 16, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 64, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 16,
   'aggregator_type': 'mean',
   'acc': 0.782423,
   'precision': 0.782423,
   'recall': 0.782423,
   'details':     epoch       loss   val_acc      time
   0       0  42.408524  0.000000  0.034891
   1       1  28.628466  0.000000  0.023886
   2       2  15.220925  0.064815  0.018217
   3       3  10.414610  0.363757  0.016327
   4       4   9.627637  0.318783  0.018147
   ..    ...        ...       ...       ...
   95     95   0.733870  0.785714  0.013995
   96     96   0.728984  0.784392  0.015428
   97     97   0.724050  0.787037  0.014964
   98     98   0.718859  0.783069  0.017076
   99     99   0.713398  0.781746  0.025292
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [40]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean', 'pool', 'lstm']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

agg_results = train_with_grid(param_grid)
agg_results

  0%|          | 0/3 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: pool
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: lstm


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.837825,
   'precision': 0.837825,
   'recall': 0.837825,
   'details':     epoch       loss   val_acc      time
   0       0  38.343391  0.017196  0.029200
   1       1  29.802776  0.017196  0.023518
   2       2  22.071142  0.158730  0.021741
   3       3  19.500887  0.244709  0.017494
   4       4  17.384188  0.265873  0.021339
   ..    ...        ...       ...       ...
   95     95   0.647309  0.841270  0.017681
   96     96   0.642519  0.839947  0.015521
   97     97   0.637861  0.845238  0.014718
   98     98   0.633823  0.850529  0.014959
   99     99   0.630021  0.851852  0.013720
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [51]:
num_hidden_layers = [3]
num_units_per_layer = [100]
activation_functions = [F.tanh]
learning_rates = [0.01]
batch_sizes = [64]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

fine_results, model = train_with_grid(param_grid)
fine_results

  0%|          | 0/1 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 100, activation: tanh, learning rate: 0.01, batch size: 64, aggreagator type: mean


[{'num_hidden_layers': 3,
  'num_units_per_layer': 100,
  'activation_functions': <function torch.nn.functional.tanh(input)>,
  'learning_rates': 0.01,
  'batch_sizes': 64,
  'aggregator_type': 'mean',
  'acc': 0.958765,
  'precision': 0.958765,
  'recall': 0.958765,
  'details':     epoch      loss   val_acc      time
  0       0  4.020436  0.038360  0.052055
  1       1  1.293679  0.396825  0.031320
  2       2  3.569330  0.376984  0.030866
  3       3  1.366555  0.433862  0.032971
  4       4  1.157817  0.466931  0.038070
  ..    ...       ...       ...       ...
  95     95  0.140882  0.964286  0.034497
  96     96  0.139538  0.961640  0.036463
  97     97  0.138030  0.966931  0.037895
  98     98  0.136848  0.961640  0.032361
  99     99  0.135836  0.961640  0.031947
  
  [100 rows x 4 columns]}]

In [53]:
dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
actual = []
for l in dictionary['alllabels']:
    actual.extend(l)

predicted = []
for l in dictionary['allpredictions']:
    predicted.extend(l)

In [54]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)
print(cf)
fig = Plotly.FigureByConfusionMatrix(cf)
Plotly.Show(fig)

[[ 160    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0  120    0    0    0    0    0    0    0    2    0    0    0]
 [   0    0  161    0    0   31    0    0    0    0    0    0    0]
 [   0    0    0   76    0   34    0    0    0    0    0    0    0]
 [   0    0    0    0  183    0    2    0    4    0    0    0    0]
 [   0    0   79  124    0  417    0    0    0    0    0    0    0]
 [   0    0    0    0   17    0  428   48    0    0    0    0    0]
 [   0    0    0    0    0    0   14  149    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0  138    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0  238    0    0    0]
 [   0    0    0    0    0    0    0    3    0    0  222    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0 3263    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0 2769]]


In [55]:
categories = set(actual+predicted)
categories
true_positives = {category: 0 for category in categories}
false_positives = {category: 0 for category in categories}
false_negatives = {category: 0 for category in categories}
n_labels = {category: 0 for category in categories}


for i in range(len(predicted)):
    if predicted[i] == actual[i]:
        true_positives[actual[i]] += 1
    else:
        false_positives[predicted[i]] += 1

for i in range(len(predicted)):
    if predicted[i] == actual[i]:
        pass
    else:
        false_negatives[actual[i]] += 1

for i in range(len(categories)):
    n_labels[i] = true_positives[i] + false_negatives[i]

true_positives
false_positives

total_true_positives = sum(true_positives.values())
total_false_positives = sum(false_positives.values())
total_true_positives / (total_true_positives + total_false_positives)

0.958765261460493

In [56]:
for c in range(len(categories)):
    if (true_positives[c] + false_positives[c]) == 0:
        precision = 0
    else:
        precision = true_positives[c] / (true_positives[c] + false_positives[c])

    if (true_positives[c] + false_negatives[c]) == 0:
        recall = 0
    else:
        recall = true_positives[c] / (true_positives[c] + false_negatives[c])
    # if (true_positives[c] + n_labels[c]) == 0:
    #     accuracy = 0
    # else:
    #     accuracy = true_positives[c]/ (n_labels[c])
    
    # print(f"accuracy for {c}: {accuracy}")
    print(c)
    print(f"precision for {c}: {precision}")
    print(f"recall for {c}: {recall}")
    print("---")

0
precision for 0: 1.0
recall for 0: 1.0
---
1
precision for 1: 0.9836065573770492
recall for 1: 1.0
---
2
precision for 2: 0.8385416666666666
recall for 2: 0.6708333333333333
---
3
precision for 3: 0.6909090909090909
recall for 3: 0.38
---
4
precision for 4: 0.9682539682539683
recall for 4: 0.915
---
5
precision for 5: 0.6725806451612903
recall for 5: 0.8651452282157677
---
6
precision for 6: 0.8681541582150102
recall for 6: 0.963963963963964
---
7
precision for 7: 0.9141104294478528
recall for 7: 0.745
---
8
precision for 8: 1.0
recall for 8: 0.971830985915493
---
9
precision for 9: 1.0
recall for 9: 0.9916666666666667
---
10
precision for 10: 0.9866666666666667
recall for 10: 1.0
---
11
precision for 11: 1.0
recall for 11: 1.0
---
12
precision for 12: 1.0
recall for 12: 1.0
---
